## Diagonalization of matrices

Created on Wed Feb 12 10:44:59 2020
@author: German Sinuco

Skeleton modified from 

https://www.tensorflow.org/tutorials/customization/custom_training 
https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough

Training of an RBM parametrization of the unitary matrix that diagonalises the 2x2 real, and symmetric HAMILTONIAN:

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import math as m
from scipy.stats import unitary_group

## Diagonalisation of a 2x2 matrix

Here we use `Tensor`, `Variable`, and `GradientTape` to build and train a model to diagonalise a 2x2 matix. This involves a few steps:

1. Define the model: define the matrix, H, and the unitary transformation that diagonalises it, U
2. Define a loss function: as the sum of the square of the offdiagonnal elements. Include a Lagrange multiplier to preserve unitarity.
4. Loop through the loss function and use an "optimizer" to adjust the variables to find U.

Here, the matrix is the Hamiltonian of a qubit in a magnetic field:

H =  delta S_z + Omega/2 Sx

The unitary transformation U, leads to the diagonal form:

H = U^dagger H U

and we parametrise the elements of the matrix:

U_ji = sqrt(P(x)_(b,c,W)/Z)

with:
    
x = j,i
 
and
    
P(x)_(b,c,W) = exp(b_ji . x) Prod_l=1^M 2 x cosh(c_l + W_{x,l} . x)

Z = sum_x P(x)_(b,c,W)

### Define the model

The model defines the Hamiltonian and the self call

In [3]:
class Model(object):
  def __init__(self):

    self.delta = 1.00
    self.Omega = 0.10

    # Initialize the spin value and number of floquet channels
    self.S   = 4                   # spin
    self.N   = 0                   # floquet manifolds
    self.dim = self.S*(2*self.N+1) # dimension of the extended floquet space
    uf_      = tf.random.stateless_uniform([self.dim,self.dim],seed=[1,1],dtype=tf.float32,minval=0.0,maxval=1.0)      
    s,u,v    = tf.linalg.svd(uf_, full_matrices=True)
    uf_      = u
    #U_seed   = tf.constant([[ 1.0095158  ,-0.01091278], [ 0.07051096 , 1.0103464 ]],dtype=tf.float32)
    #U_seed   = tf.constant([[ 0.999688 , -0.0249766], [ 0.0249766 , 0.999688 ]],dtype=tf.float32)
    #self.UF  = tf.Variable(U_seed,dtype = tf.float32,trainable = True)  # ext. micromotion operator
    self.UF  = tf.Variable(uf_,dtype = tf.float32,trainable = True)  # ext. micromotion operator
    
    # spin 1/2 
    #Identity   =     tf.constant([[1.0,0.0],[ 0.0, 1.0]],dtype = tf.float32)
    #Sx         = 0.5*tf.constant([[0.0,1.0],[ 1.0, 0.0]],dtype = tf.float32)
    #Sz         = 0.5*tf.constant([[1.0,0.0],[ 0.0,-1.0]],dtype = tf.float32)

    # spin 3/2
    Identity   =     tf.constant([[1.0,0.0,0.0,0.0],
                                  [0.0,1.0,0.0,0.0],
                                  [0.0,0.0,1.0,0.0],
                                  [0.0,0.0,0.0,1.0]],dtype = tf.float32)
    Sx         = 0.5*tf.constant([[0.0,         np.sqrt(3.0),0.0,          0.0],
                                  [np.sqrt(3.0),0.0,         np.sqrt(4.0), 0.0],
                                  [0.0,         np.sqrt(4.0),0.0,          np.sqrt(4.0)],
                                  [0.0,         0.0,         np.sqrt(3.0), 0.0]],dtype = tf.float32)
    Sz         = 0.5*tf.constant([[3.0,0.0, 0.0, 0.0],
                                  [0.0,1.0, 0.0, 0.0],
                                  [0.0,0.0,-1.0, 0.0],
                                  [0.0,0.0, 0.0,-3.0]],dtype = tf.float32)

    
    
    self.H_TLS = tf.Variable(self.delta*Sz+0.5*self.Omega*Sx,shape=(self.dim,self.dim),dtype = tf.float32,trainable = False)  # ext. Hamiltonian
        
    self.trainable_variables = self.UF
    
  def getH(self):
    return self.H_TLS
    
  def __call__(self):     
      # The Gram-Schmidt process should go here because the gradient looks into this function.
      #print(Gram_Schmidt(self.UF))
      #print(self.UF)
      #UF = tf.Variable(np.empty((self.dim,self.dim), dtype=np.float32))
      #UF.assign(Gram_Schmidt(self.UF))
      #print(self.UF)
      #return tf.square(tf.abs(tf.transpose(self.UF)@(self.H_TLS@self.UF)))
      #print(self.H_TLS)
      #print(self.UF)
      #U_ = self.H_TLS@UF
      return self.H_TLS
        
      #trace_   = tf.linalg.trace(H__)/(self.dim*self.dim)      
      #residual = tf.sqrt(tf.reduce_mean(H__) - trace_)      
      #return residual
  

### Define a loss function and its gradient

A loss function measures how well the output of a model for a given input matches the target output. The goal is to minimize this difference during training. Let's use the standard L2 loss, also known as the least square errors:

In [4]:
def normalisation(U_):
    # U_  (in) original matrix
    #     (out)  matrix with normalised vectors
    normaU_ = tf.sqrt(tf.math.reduce_sum(tf.multiply(U_,U_,1),axis=0))
    U_ = tf.math.truediv(U_,normaU_)
    return U_

def Gram_Schmidt(U_):
    U_ = normalisation(U_)
    Y = []
    for i in range(0,tf.shape(U_).numpy()[0]):
        
        temp_vec = U_[:,i]
        
        for inY in Y:
            
            norminY    = tf.math.reduce_sum(tf.multiply(inY,inY,0)).numpy()
            projec_vec = tf.math.multiply(tf.math.reduce_sum(tf.multiply(U_[:,i],inY,0))/norminY,inY)
            temp_vec   = tf.math.subtract(temp_vec,projec_vec)
              
        temp_vec = normalisation(temp_vec)     
        Y.append(temp_vec)    

    Y = np.transpose(np.stack(Y))
    return Y

def train(model,learning_rate):
  with tf.GradientTape() as t:
    current_loss = loss(model)
  dU = t.gradient(current_loss, model.trainable_variables)
  model.UF.assign_sub(learning_rate*dU)
    #where should I obtain the unitary matrix? after this step?
    # should we apply a constrain as the one in the finantial applications?

In [5]:
# Here we now the predicted and the target
#def loss(predicted_y, target_y):
#  return tf.reduce_mean(tf.square(predicted_y - target_y))

# Here we now the model and the input. Then we evaluate the predicted_y
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# First attemp to a loss function. 
# 1. Evaluate the transformed Hamiltonian. 
# 2. Calculate the average of the off-diagonal elements
#def loss(model):
#  predicted_H = model()
#  #print(predicted_H)
#  trace_   = tf.linalg.trace(predicted_H)/(model.dim*model.dim)      
#  #print(trace_)
#  residual = tf.sqrt(tf.reduce_mean(predicted_H) - trace_)      
#  #print(residual)
#  return residual

# Second attemp to a loss function. 
# 1. Make sure the column vectors are normalised to 1 (in the model).
# 2. Evaluate the action of the Hamiltonian over the column-vectors of the transformation matrix
# 3a. Calculate the average of the magnitude of the cross section between the column-vectors 
#    and the obtained ones after applying the Hamiltonian?
#    return  < |H() X ()|^2 >_{all column vectors}
# 3b. can we use the dot product normalised to the product of the magnitude of the vectors?
# 3c. can we use skew product, multilinear algebra explained in the wikipedia cross product website?
# 3d. use ortonormalisaton process to loop through the unitary transformationss
#     use change of basis
# 3e. none of the above worked. Use U^dagger H U, sum over the columns, take the difference with the diagonal, 
#      the loss function is the summ of the square of these differences. 
#def loss(model):
#  predicted_H = model()
#  #print(predicted_H)
#  trace_   = tf.linalg.trace(predicted_H)/(model.dim*model.dim)      
#  #print(trace_)
#  residual = tf.sqrt(tf.reduce_mean(predicted_H) - trace_)      
#  #print(residual)
#  return residual

def loss(model):
  # maybe I should execute the Gram-Schmidt proccess here?
  U_ = tf.abs(tf.transpose(model.UF)@model.H_TLS@model.UF) #model()
  U_diag = tf.linalg.tensor_diag_part(U_)  
  #print(U_diag)
  #print(U_)
  # norm of column vectors in U
  #normaU  = tf.sqrt(tf.math.reduce_sum(tf.tensordot(tf.transpose(model.UF) ,model.UF ,1),axis=1,))
  # norm of column vectors in U_ = H U
  normaU_ = tf.sqrt(tf.math.reduce_sum(tf.tensordot(tf.transpose(U_),U_,1),axis=1,))
  #print(normaU)
  #print(normaU_)
  # product of the norms of the column vectors in U and U_ =  H U
  normaProd = 1.0#normaU_#tf.multiply(normaU,normaU_)
  #print(normaProd)
  # dot product between column vectors in U and U_ = H U
  H_  = U_#tf.abs(tf.transpose(model.UF)@U_)#tf.abs(tf.tensordot(tf.transpose(model.UF),U_,1))# tf.multiply(model.UF,U_)#
  #print(H_)
  dotProd = tf.math.reduce_sum(H_,axis=1,)
  #print(dotProd)
  projections = tf.multiply(dotProd,1/normaProd)
  #print(projections)
  residual = tf.math.reduce_sum(tf.pow((U_diag-projections),2),0)
  #residual = tf.math.reduce_sum(tf.abs((1-projections)),0)
  #residual = tf.math.reduce_sum(tf.abs((1-projections)),0)

  return residual


# This is the gradient of the loss function. required for keras optimisers
def grad(model):
  with tf.GradientTape() as tape:
    loss_value = loss(model)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)


model = Model()
# Collect the history of W-values and b-values to plot later
#Us = [], []

epochs = range(4096)
current_loss = loss(model)
print(current_loss.numpy())
#print(grad(model))
print(model.UF.numpy())
print(tf.transpose(model.UF)@(model.H_TLS@model.UF))

learning_rate = 0.01
for i in epochs:
    with tf.GradientTape() as t:
        current_loss = loss(model)
        dU = t.gradient(current_loss, model.trainable_variables)
    #print(dU)
    #print(dU)
    model.UF.assign_sub(learning_rate*dU)
    model.UF.assign(Gram_Schmidt(model.UF))

    #print(model.UF.numpy())
    current_loss = loss(model)
    #print(current_loss.numpy())
print(current_loss.numpy())
print(model.UF.numpy())
print(tf.transpose(model.UF)@(model.H_TLS@model.UF))
 
           

W0229 17:08:38.811374 139796177667904 deprecation.py:323] From /home/german/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


7.5998726
[[ 0.28523198 -0.1065064  -0.8796557  -0.36538368]
 [ 0.84665966 -0.28533936  0.13056783  0.42976823]
 [ 0.29298386  0.95226645 -0.04939937  0.07006439]
 [ 0.34054422 -0.0206559   0.4546623  -0.82273024]]
tf.Tensor(
[[ 0.31860572 -0.25286624 -0.57098126  0.42576036]
 [-0.2506534  -0.42269626  0.19839343 -0.07903269]
 [-0.5719863   0.19549999  0.8452311   1.0573735 ]
 [ 0.42753494 -0.07379422  1.0573146  -0.7411402 ]], shape=(4, 4), dtype=float32)
2.5511592e-05
[[-4.3222982e-02  1.0215507e-03 -9.9906492e-01 -2.5318534e-05]
 [ 9.9781805e-01 -4.9895044e-02 -4.3220021e-02 -1.3099322e-03]
 [ 4.9899373e-02  9.9767166e-01 -1.1398697e-03  4.6475809e-02]
 [ 1.0142307e-03  4.6483196e-02  2.8965123e-05 -9.9891859e-01]]
tf.Tensor(
[[ 5.00624955e-01  2.59336084e-05 -1.07958913e-05 -4.65171644e-04]
 [ 1.71735883e-05 -5.00324547e-01  1.16538955e-04 -3.41914594e-03]
 [-1.08033419e-05  1.15990173e-04  1.50187373e+00  1.49485437e-04]
 [-1.30954664e-04  3.27123329e-03  1.41867014e-04 -1.5021745

### Exact solution

First, build the exact unitary transformation and perturb it

In [6]:
Sx    = 0.5*tf.constant([[0,1],[ 1, 0]],dtype = tf.float32)
Sz    = 0.5*tf.constant([[1,0],[ 0,-1]],dtype = tf.float32)   
delta = 1.0
Omega = 0.1

H_TLS = delta*Sz+0.5*Omega*Sx
theta = 0.5*np.arctan(0.25*Omega/(-0.5*delta))
U     = [[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]]
U     = tf.convert_to_tensor(U, np.float32)
noise = tf.random.stateless_uniform([2,2],seed=[1,1],dtype=tf.float32,minval=0.0,maxval=0.1)      
U     = U #+ noise

print(H_TLS)
print(U)

print(H_TLS@U)

print(tf.transpose(U)@(H_TLS@U))



tf.Tensor(
[[ 0.5    0.025]
 [ 0.025 -0.5  ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 0.999688  -0.0249766]
 [ 0.0249766  0.999688 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 0.50046843  0.0125039 ]
 [ 0.0125039  -0.50046843]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 0.50062466  0.        ]
 [ 0.         -0.50062466]], shape=(2, 2), dtype=float32)


### Define a training loop

With the network and training data, train the model using [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) to update the matrix 'U' to reduce the loss. There are many variants of the gradient descent scheme that are captured in `tf.train.Optimizer`—our recommended implementation. But in the spirit of building from first principles, here you will implement the basic math yourself with the help of `tf.GradientTape` for automatic differentiation and `tf.assign_sub` for decrementing a value (which combines `tf.assign` and `tf.sub`):

In [7]:
def normalisation(U_):
    # U_  (in) original matrix
    #     (out)  matrix with normalised vectors
    normaU_ = tf.sqrt(tf.math.reduce_sum(tf.multiply(U_,U_,1),axis=0))
    U_ = tf.math.truediv(U_,normaU_)
    return U_

def Gram_Schmidt(U_):
    U_ = normalisation(U_)
    Y = []
    for i in range(0,tf.shape(U_).numpy()[0]):
        
        temp_vec = U_[:,i]
        
        for inY in Y:
            
            norminY    = tf.math.reduce_sum(tf.multiply(inY,inY,0)).numpy()
            projec_vec = tf.math.multiply(tf.math.reduce_sum(tf.multiply(U_[:,i],inY,0))/norminY,inY)
            temp_vec   = tf.math.subtract(temp_vec,projec_vec)
              
        temp_vec = normalisation(temp_vec)     
        Y.append(temp_vec)    

    Y = np.transpose(np.stack(Y))
    return Y

def train(model,learning_rate):
  with tf.GradientTape() as t:
    current_loss = loss(model)
  dU = t.gradient(current_loss, model.trainable_variables)
  model.UF.assign_sub(learning_rate*dU)
    #where should I obtain the unitary matrix? after this step?
    # should we apply a constrain as the one in the finantial applications?

Finally, let's repeatedly run through the training data and see how `W` and `b` evolve.

In [8]:
model = Model()

# Collect the history of W-values and b-values to plot later
Us = [], []

epochs = range(1)
current_loss = loss(model)
print(current_loss.numpy())

#for epoch in epochs:
  #Us.append(model.UF.numpy())  
  #print(model.UF.numpy())
  #current_loss = loss(model)
  #print(current_loss.numpy())
  #train(model, learning_rate=1.0)

#print(current_loss.numpy())
#print(model.UF.numpy())
  

7.5998726
